In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.3.2`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = SparkSession.builder()
  .appName("SparkTest")
  .master("local[*]")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark is ready!")

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark is ready!


import $ivy.$                                      

import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@3562b6a6

In [2]:
import spark.implicits._

case class Employee(employee_id: Int, name: String, reports_to: Option[Int], age: Int)

val employee: Seq[(Int, String, Option[Int], Int)] = Seq(
  (9, "Hercy", None, 43),
  (6, "Alice", Some(9), 41),
  (4, "Bob", Some(9), 36),
  (2, "Winston", None, 37)
)

val employeesDS = employee.toDF("employee_id", "name", "reports_to", "age").as[Employee]


import spark.implicits._


defined class Employee
employee: Seq[(Int, String, Option[Int], Int)] = List(
  (9, "Hercy", None, 43),
  (6, "Alice", Some(value = 9), 41),
  (4, "Bob", Some(value = 9), 36),
  (2, "Winston", None, 37)
)
employeesDS: Dataset[Employee] = [employee_id: int, name: string ... 2 more fields]

For this problem, we will consider a manager an employee who has at least 1 other employee reporting to them.

Write a solution to report the ids and the names of all managers, the number of employees who report directly to them, and the average age of the reports rounded to the nearest integer.

Return the result table ordered by employee_id.

The result format is in the following example.

| employee_id | name  | reports_count | average_age |
|-------------|-------|---------------|-------------|
| 9           | Hercy | 2             | 39          |

In [3]:
employeesDS.show()

val managersDF = employeesDS.select("reports_to", "age").filter(col("reports_to").isNotNull)
    .groupBy(col("reports_to"))
    .agg(round(avg(col("age")), 0).cast("int").as("average_age"), count(lit(1)).as("reports_count"))

managersDF.as("a").join(employeesDS.as("b"), col("b.employee_id") === col("a.reports_to"), "inner")
    .select("employee_id", "name", "reports_count", "average_age")
    .show()

+-----------+-------+----------+---+
|employee_id|   name|reports_to|age|
+-----------+-------+----------+---+
|          9|  Hercy|      null| 43|
|          6|  Alice|         9| 41|
|          4|    Bob|         9| 36|
|          2|Winston|      null| 37|
+-----------+-------+----------+---+

+-----------+-----+-------------+-----------+
|employee_id| name|reports_count|average_age|
+-----------+-----+-------------+-----------+
|          9|Hercy|            2|         39|
+-----------+-----+-------------+-----------+



managersDF: DataFrame = [reports_to: int, average_age: int ... 1 more field]

In [4]:
spark.stop()